# 파트 6: 결합된 지식 소스

파트 1-5에서는 다양한 유형의 지식 소스를 사용했습니다: 검색 인덱스(파트 1-2), SharePoint(파트 3), 웹 소스(파트 4), blob 스토리지(파트 5). 파트 6에서는 **다양한 유형**의 지식 소스를 결합하고 전체 데이터 에코시스템을 동시에 쿼리할 수 있는 통합 인터페이스를 생성합니다.

## 단계 1: 환경 변수 로드

아래 셀을 실행하여 Azure 리소스의 구성을 로드하고, 생성된 **.venv(3.11.9)** 환경을 선택하세요.

이번에는 탐색한 모든 다양한 유형의 지식 소스를 참조하는 지식 베이스를 생성합니다.

> **⚠️ 문제 해결**
>
> 코드 셀이 멈추고 계속 돌아가면 노트북 상단 툴바에서 **Restart**를 선택하세요. 몇 번 시도해도 문제가 지속되면 VS Code를 완전히 닫고 다시 여세요.

In [1]:
import os

from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

load_dotenv(override=True) # take environment variables from .env.

# Azure AI Search configuration
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"])

# Knowledge base name
knowledge_base_name = "combined-knowledge-base"

# Azure OpenAI configuration
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
azure_openai_chatgpt_deployment = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT", "gpt-4.1")
azure_openai_chatgpt_model_name = os.getenv("AZURE_OPENAI_CHATGPT_MODEL_NAME", "gpt-4.1")

print("Environment variables loaded")

Environment variables loaded


## 단계 2: 다양한 소스 유형에 걸친 통합 지식 베이스 생성

이제 다양한 유형의 지식 소스를 결합하는 지식 베이스를 생성합니다: 검색 인덱스(`healthdocs-knowledge-source` 및 `hrdocs-knowledge-source`)와 웹 소스(`web-knowledge-source`).

아래 코드를 실행하여 하나의 통합 인터페이스에서 다양한 소스 유형(내부 인덱싱된 데이터와 외부 웹 콘텐츠)을 쿼리합니다.

In [2]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import AzureOpenAIVectorizerParameters, KnowledgeBase, KnowledgeBaseAzureOpenAIModel, KnowledgeRetrievalOutputMode, KnowledgeSourceReference, WebKnowledgeSource

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=azure_openai_endpoint,
    api_key=azure_openai_key,
    deployment_name=azure_openai_chatgpt_deployment,
    model_name=azure_openai_chatgpt_model_name,
)

ks = WebKnowledgeSource(
    name="web-knowledge-source",
    description="Knowledge source for Web"
)
index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{ks.name}' created or updated successfully.")

knowledge_base = KnowledgeBase(
    name=knowledge_base_name,
    models=[KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(name="healthdocs-knowledge-source"),
        KnowledgeSourceReference(name="hrdocs-knowledge-source"),
        KnowledgeSourceReference(name="web-knowledge-source"),
    ],
    output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS,
    answer_instructions="답변을 제공할 때는 항상 글머리 기호 목록 형식을 사용하세요. 각 항목은 별도의 줄에 표시해야 합니다. 인용한 출처의 이름을 해당 줄 앞에 추가하세요.",
)

index_client.create_or_update_knowledge_base(knowledge_base)
print(f"Knowledge base '{knowledge_base_name}' created or updated successfully.")

Knowledge source 'web-knowledge-source' created or updated successfully.
Knowledge base 'combined-knowledge-base' created or updated successfully.


## 단계 3: 내부 주제에 대해 다중 소스 유형 쿼리

아래 코드는 하나의 쿼리에서 두 가지 질문을 합니다: Zava CEO에 대한 질문(HR 정보)과 건강 플랜에 대한 질문(건강 혜택 정보). 이는 결합된 지식 베이스가 다양한 내부 데이터 소스에 걸친 쿼리를 어떻게 처리하는지 보여줍니다.

지식 베이스는 HR 문서, 건강 문서, 웹 소스를 검색한 다음, 각 소스 유형에서 적절한 인용과 함께 두 질문을 모두 다루는 포괄적인 답변을 합성합니다.

In [5]:
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import KnowledgeBaseMessage, KnowledgeBaseMessageTextContent, KnowledgeBaseRetrievalRequest, SearchIndexKnowledgeSourceParams, WebKnowledgeSourceParams
from IPython.display import display, Markdown

knowledge_base_client = KnowledgeBaseRetrievalClient(endpoint=endpoint, knowledge_base_name=knowledge_base_name, credential=credential)
web_ks_params = WebKnowledgeSourceParams(
    knowledge_source_name="web-knowledge-source",
    include_references=True,
    include_reference_source_data=True
)
healthdocs_ks_params = SearchIndexKnowledgeSourceParams(
    knowledge_source_name="healthdocs-knowledge-source",
    include_references=True,
    include_reference_source_data=True,
)
hrdocs_ks_params = SearchIndexKnowledgeSourceParams(
    knowledge_source_name="hrdocs-knowledge-source",
    include_references=True,
    include_reference_source_data=True,
)
req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="""
            Zava CEO의 책임은 무엇인가요?
            정신 건강 서비스에 대한 최고의 보장을 원한다면 어떤 Zava 건강 보험 플랜을 추천하시겠습니까?                      
        """)])
    ],
    knowledge_source_params=[
        web_ks_params, healthdocs_ks_params, hrdocs_ks_params
    ],
    include_activity=True
)

result = knowledge_base_client.retrieve(retrieval_request=req)
display(Markdown(result.response[0].content[0].text))

Zava CEO의 책임:
Azure Search (hrdocs):
• 회사의 장기적인 성공과 수익성을 보장하기 위해 전략과 목표를 개발 및 구현합니다 [ref_id:2].
• 임원진에게 전략적 지침을 제공하고 회사의 일상 운영을 관리합니다 [ref_id:2].
• 모든 법률 및 규정 준수를 보장합니다 [ref_id:2].
• 마케팅 및 광고 전략 개발을 감독합니다 [ref_id:2].
• 주요 이해관계자와의 관계를 개발 및 관리합니다 [ref_id:2].
• 대외 행사에서 회사를 대표합니다 [ref_id:2].
• 공급업체 및 벤더와의 계약을 관리 및 협상합니다 [ref_id:2].
• 업계 동향을 모니터링하고 성장 기회를 식별합니다 [ref_id:2].
• 긍정적인 기업 문화를 개발 및 유지합니다 [ref_id:2].

정신 건강 서비스에 대한 최고의 보장 플랜 추천:
Azure Search (healthdocs):
• Northwind Health Plus 플랜은 의료, 시력, 치과, 처방약, 정신 건강 및 예방 진료 등 포괄적인 보장을 제공합니다 [ref_id:8].
• 이 플랜은 인네트워크 및 아웃오브네트워크 제공자를 모두 사용할 수 있으며, 인네트워크 이용 시 본인 부담금이 더 낮고 보장이 더 넓습니다 [ref_id:8][ref_id:3].
• 정신 건강 서비스에는 상담, 정신과 방문, 치료, 그룹 치료 등이 포함되며, 입원 및 외래 모두 보장됩니다 [ref_id:3].
• 정신 건강 보장 범위와 비용 부담 측면에서 Northwind Health Plus가 가장 포괄적이므로, 최고의 정신 건강 보장을 원한다면 이 플랜을 추천합니다 [ref_id:8][ref_id:3].

## 단계 4: 웹 전용 주제 쿼리

다음 쿼리는 결합된 지식 베이스가 질문을 적절한 소스로 지능적으로 라우팅하는 방법을 보여줍니다. 일반적인 세계 지식(에펠탑과 같은)에 대해 질문하면, 내부 HR 및 건강 문서에는 이 정보가 없으므로 시스템은 주로 웹 지식 소스를 사용합니다.

이는 모든 소스를 사용할 수 있는 경우에도 지식 베이스가 다양한 유형의 쿼리에 가장 관련 있는 소스를 이해하는 능력을 보여줍니다.

In [6]:
req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="""
            한국 마이크로소프트의 주소는?                    
        """)])
    ],
    knowledge_source_params=[
        web_ks_params, healthdocs_ks_params, hrdocs_ks_params
    ],
    include_activity=True
)

result = knowledge_base_client.retrieve(retrieval_request=req)
display(Markdown(result.response[0].content[0].text))

- Web (네이버 블로그): 서울 종로구 종로1길 50 더케이트윈타워 A동 11층[ref_id:0]
- Web (bizkorea.org): 서울 종로구 중학동 19번지 케이트윈타워 A동 12층[ref_id:1]
- Web (support.microsoft.com): (우)110-150 서울 종로구 종로1길 50 더 케이트윈타워 A동 12층[ref_id:2]
- Web (bizno.net): 서울특별시 종로구 종로1길 50 (중학동, 더케이트윈타워 에이동 11층,12층,13층,14층,15층,16층)[ref_id:4]
- Web (incruit.com): 서울특별시 종로구 종로1길 50 A동 12층 (중학동,케이트윈타)[ref_id:6]

## 요약

이제 다양한 유형의 데이터 소스에 걸친 단일 지식 베이스에서 결합된 지식 소스를 사용했습니다.

**기억해야 할 핵심 개념:**
- 단일 지식 베이스는 다양한 유형의 지식 소스(검색 인덱스, 웹 소스)를 참조할 수 있습니다
- 지식 베이스는 콘텐츠에 따라 쿼리를 적절한 소스로 지능적으로 라우팅합니다
- 결합된 소스는 적절한 소스 귀속과 함께 포괄적인 답변을 제공합니다
- 시스템은 어떤 지식 소스를 쿼리할지 수동으로 결정할 필요성을 제거합니다

### 다음 단계

➡️ [파트 7: 최소 지식 베이스](part7-minimal-knowledge-base.ipynb)로 계속하여 최소 추론 노력으로 속도와 비용을 최적화하는 방법을 배웁니다.